In [ ]:

import cv2
import os
from PIL import Image

# Function to convert an image to thermal
def convert_to_thermal(image_path):
    image = cv2.imread(image_path)
    thermal_image = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    return Image.fromarray(thermal_image)

# Convert all images in a directory to thermal images
def convert_directory_to_thermal(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                # Create output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                thermal_image = convert_to_thermal(input_path)
                thermal_image.save(output_path)

# Paths to original and thermal image directories
train_dir = "/path/to/train"  # Replace with the actual path to train folder
test_dir = "/path/to/test"    # Replace with the actual path to test folder
val_dir = "/path/to/val"      # Replace with the actual path to val folder

thermal_train_dir = "/path/to/thermal_train"  # Replace with the path where thermal images will be saved
thermal_test_dir = "/path/to/thermal_test"    # Replace with the path where thermal images will be saved
thermal_val_dir = "/path/to/thermal_val"      # Replace with the path where thermal images will be saved

# Convert the datasets to thermal images
convert_directory_to_thermal(train_dir, thermal_train_dir)
convert_directory_to_thermal(test_dir, thermal_test_dir)
convert_directory_to_thermal(val_dir, thermal_val_dir)


In [ ]:
!kaggle datasets download -d sshikamaru/lyme-disease-rashes

Dataset URL: https://www.kaggle.com/datasets/sshikamaru/lyme-disease-rashes
License(s): copyright-authors
 89% 127M/143M [00:01<00:00, 98.6MB/s]
100% 143M/143M [00:01<00:00, 107MB/s] 


In [ ]:
! unzip 'lyme-disease-rashes'

Streaming output truncated to the last 5000 lines.
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash168.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash169.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash17.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash170.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash171.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash172.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash173.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash174.jpg 

In [ ]:
TRAIN_DIR_ensemble = '/content/Lyme Full Statified/Lyme Full Statified/image_folder_balanced/image_folder_balanced/train_balanced'
TEST_DIR_ensemble ='/content/Lyme Full Statified/Lyme Full Statified/image_folder_balanced/image_folder_balanced/val_balanced'

In [ ]:
TRAIN_DIR = '/content/RashData/RashData/Train/Train_2_Cases'
TEST_DIR = '/content/RashData/RashData/Validation/Validation_2_Cases'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input


HEIGHT = 300
WIDTH = 300

BATCH_SIZE = 8
class_list = ["class_1", "class_2"]
FC_LAYERS = [1024, 512, 256]
drop = 0.5
EPOCHS = 100

#RESNET

In [ ]:
train_datagen= ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    horizontal_flip=True,
    zoom_range=0.1
)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  rotation_range = 90,

                                  vertical_flip = False)

In [ ]:
train_generator= train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (HEIGHT, WIDTH),
                                                    batch_size = BATCH_SIZE)

test_generator= test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size = (HEIGHT, WIDTH),
                                                  batch_size = BATCH_SIZE)

Found 357 images belonging to 2 classes.
Found 87 images belonging to 2 classes.


#ensemble

In [ ]:
train_datagen_ensemble = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    horizontal_flip=True,
    zoom_range=0.1
)

test_datagen_ensemble = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  rotation_range = 90,
                                  vertical_flip = False)

In [ ]:
train_generator_ensemble = train_datagen_ensemble.flow_from_directory(TRAIN_DIR_ensemble,
                                                    target_size = (HEIGHT, WIDTH),
                                                    batch_size = BATCH_SIZE)

test_generator_ensemble = test_datagen_ensemble.flow_from_directory(TEST_DIR_ensemble,
                                                  target_size = (HEIGHT, WIDTH),
                                                  batch_size = BATCH_SIZE)

Found 1506 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


#transfer learning with EfficientNet

In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, BatchNormalization

In [ ]:
backbone=tf.keras.applications.efficientnet.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(HEIGHT,WIDTH,3)
)

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
backbone.trainable=False

In [ ]:
# Apply the backbone model
inputs = Input(shape=(HEIGHT, WIDTH, 3))
x = backbone(inputs)

# Add GlobalAveragePooling2D layer
x = GlobalAveragePooling2D()(x)

# Add Dense and BatchNormalization layers
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)

# Define the output layer
outputs = Dense(2, activation='softmax')(x)

# Create the functional model
model1 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model1.compile(optimizer=Adam(learning_rate=1e-5), loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
history = model1.fit(
    train_generator,
    epochs=100,
    steps_per_epoch=20,
    validation_data=test_generator,
    shuffle=True
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5626 - loss: 0.7621

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.5625 - loss: 0.7617 - val_accuracy: 0.5172 - val_loss: 0.6884
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 340ms/step - accuracy: 0.5664 - loss: 0.7363 - val_accuracy: 0.5632 - val_loss: 0.6819
Epoch 3/100
 5/20 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4825 - loss: 0.7999

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.4769 - loss: 0.7813 - val_accuracy: 0.6437 - val_loss: 0.6696
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 62s 325ms/step - accuracy: 0.6010 - loss: 0.6857 - val_accuracy: 0.6207 - val_loss: 0.6651
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 331ms/step - accuracy: 0.5372 - loss: 0.7310 - val_accuracy: 0.5517 - val_loss: 0.6782
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - accuracy: 0.6333 - loss: 0.6504 - val_accuracy: 0.6782 - val_loss: 0.6650
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 365ms/step - accuracy: 0.5973 - loss: 0.7210 - val_accuracy: 0.6322 - val_loss: 0.6678
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 262ms/step - accuracy: 0.5652 - loss: 0.6703 - val_accuracy: 0.6092 - val_loss: 0.6675
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.6377 - loss: 0.6782 - val_accuracy: 0.6667 - val_loss: 0.6601
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 490ms/step - accuracy: 0.7011 - loss: 0.6381 - val_accuracy: 0.

In [ ]:
test_loss, test_accuracy = model1.evaluate( test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - accuracy: 0.7879 - loss: 0.4938


In [ ]:
model1.save_weights("model11_weights2.weights.h5")

#transfer learning with resnet

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(HEIGHT,WIDTH,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
base_model.trainable = False

# Start with the input and output of the base model
inputs = base_model.input
x = base_model.output

# Flatten the output if necessary
x = Flatten()(x)

# Add the fully connected (dense) layers with dropout
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)


# Final prediction layer
predictions = Dense(2, activation='softmax')(x)

# Create the final model
model2 = Model(inputs=inputs, outputs=predictions)

In [ ]:
from keras.optimizers import Adam
model2.compile(optimizer=Adam(learning_rate=1e-5), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model2.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 300, 300, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 306, 306, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 150, 150, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 150, 150, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 150, 150, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 152, 152, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 75, 75, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 75, 75, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 75, 75, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 75, 75, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 75, 75, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 75, 75, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 75, 75, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 75, 75, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 75, 75, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 233,960,578 (892.49 MB)

 Trainable params: 210,372,866 (802.51 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
history = model2.fit(
    train_generator,
    epochs=100,
    steps_per_epoch=20,
    validation_data=test_generator,
    shuffle=True
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 32s 790ms/step - accuracy: 0.5157 - loss: 1.5897 - val_accuracy: 0.4947 - val_loss: 0.8246
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 448ms/step - accuracy: 0.5661 - loss: 1.5980 - val_accuracy: 0.4842 - val_loss: 0.8065
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 428ms/step - accuracy: 0.5333 - loss: 1.5583 - val_accuracy: 0.5263 - val_loss: 0.8386
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 369ms/step - accuracy: 0.4999 - loss: 1.5309 - val_accuracy: 0.5105 - val_loss: 0.7816
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 694ms/step - accuracy: 0.5166 - loss: 1.3603 - val_accuracy: 0.5421 - val_loss: 0.7843
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 424ms/step - accuracy: 0.4937 - loss: 1.4255 - val_accuracy: 0.5842 - val_loss: 0.7127
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 389ms/step - accuracy: 0.5211 - loss: 1.2681 - val_accuracy: 0.5684 - val_loss: 0.7136
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 430ms/step - accuracy: 0.5152 - loss: 1.2574 - val_ac

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 296ms/step - accuracy: 0.5623 - loss: 1.4819 - val_accuracy: 0.4842 - val_loss: 0.7790
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 379ms/step - accuracy: 0.5323 - loss: 1.3061 - val_accuracy: 0.5895 - val_loss: 0.7030
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 459ms/step - accuracy: 0.5686 - loss: 1.2083 - val_accuracy: 0.5947 - val_loss: 0.6918
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 399ms/step - accuracy: 0.3857 - loss: 1.3641 - val_accuracy: 0.5158 - val_loss: 0.7380
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 420ms/step - accuracy: 0.5200 - loss: 1.3032 - val_accuracy: 0.5579 - val_loss: 0.7268
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 444ms/step - accuracy: 0.5390 - loss: 1.2035 - val_accuracy: 0.5526 - val_loss: 0.6857
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 365ms/step - accuracy: 0.5213 - loss: 0.9659 - val_accuracy: 0.6053 - val_loss: 0.7021
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 436ms/step - accuracy: 0.5571 - loss: 1.1099 - val_accuracy

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model2.evaluate( test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.9407 - loss: 0.2165


In [ ]:
model1.save_weights("model22(1)_weights2.weights.h5")

#transfer learning with vision Transformer




In [ ]:
!pip install transformers

In [ ]:
#!pip install --upgrade transformers  # Ensure you have the latest version
from transformers import ViTFeatureExtractor, TFViTModel  # Check the class name spelling

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Rescaling, Permute, Layer
from tensorflow.keras.models import Model
from transformers import TFViTModel

# Define the preprocessing layers
resize_rescale_hf = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),
    Rescaling(1./255),
    Permute((3, 1, 2))
])

In [ ]:
class ViTLayer(Layer):
    def __init__(self, base_model):
        super(ViTLayer, self).__init__()
        self.base_model = base_model

    def call(self, inputs):
        return self.base_model.vit(inputs)[0][:, 0, :]

In [ ]:
from tensorflow.keras.layers import BatchNormalization
# Load the pre-trained ViT model
base_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False
# Define the input layer
inputs = Input(shape=(300, 300, 3))
x = resize_rescale_hf(inputs)
x = ViTLayer(base_model)(x)
x = Dense(1024,activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(128,activation='relu')(x)
x = BatchNormalization()(x)

output = Dense(2, activation='softmax')(x)

# Create the Keras model
hf_model = Model(inputs=inputs, outputs=output)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy
metrics=[CategoricalAccuracy(name='accuracy')]

In [ ]:
hf_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=metrics
)

In [ ]:
history=hf_model.fit(
    train_generator,
    epochs=100,
    steps_per_epoch=20,
    validation_data=test_generator,
    shuffle=True
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 873ms/step - accuracy: 0.5400 - loss: 0.8319 - val_accuracy: 0.5474 - val_loss: 0.6943
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 441ms/step - accuracy: 0.5421 - loss: 0.8648 - val_accuracy: 0.4947 - val_loss: 0.6953
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 718ms/step - accuracy: 0.6026 - loss: 0.7813 - val_accuracy: 0.5316 - val_loss: 0.6896
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 412ms/step - accuracy: 0.6175 - loss: 0.7298 - val_accuracy: 0.5211 - val_loss: 0.6869
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 369ms/step - accuracy: 0.5621 - loss: 0.8855 - val_accuracy: 0.5105 - val_loss: 0.6852
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 473ms/step - accuracy: 0.6285 - loss: 0.6950 - val_accuracy: 0.5211 - val_loss: 0.6965
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 360ms/step - accuracy: 0.6947 - loss: 0.6398 - val_accuracy: 0.5263 - val_loss: 0.6999
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 427ms/step - accuracy: 0.6269 - loss: 0.7404 - val_accuracy: 0.58

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 309ms/step - accuracy: 0.6060 - loss: 0.7553 - val_accuracy: 0.5474 - val_loss: 0.6915
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 54s 442ms/step - accuracy: 0.6329 - loss: 0.7198 - val_accuracy: 0.5579 - val_loss: 0.6765
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 459ms/step - accuracy: 0.6567 - loss: 0.6858 - val_accuracy: 0.5474 - val_loss: 0.6791
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 468ms/step - accuracy: 0.7202 - loss: 0.6344 - val_accuracy: 0.5947 - val_loss: 0.6813
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 424ms/step - accuracy: 0.7130 - loss: 0.6238 - val_accuracy: 0.6263 - val_loss: 0.6700
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 450ms/step - accuracy: 0.6934 - loss: 0.6040 - val_accuracy: 0.6263 - val_loss: 0.6776
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 635ms/step - accuracy: 0.6273 - loss: 0.7235 - val_accuracy: 0.6211 - val_loss: 0.6788
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 634ms/step - accuracy: 0.7211 - loss: 0.6180 - val_accura

In [ ]:
test_loss, test_accuracy = hf_model.evaluate( test_generator_vision)

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.6024 - loss: 0.8473


In [ ]:
# Save the model weights
hf_model.save_weights("model33_weights2.weights.h5")

#Training the ensemble with the labels

In [ ]:
print(model2.summary())

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 300, 300, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 306, 306, 3)    │              0 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 150, 150, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 150, 150, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 150, 150, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 152, 152, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 75, 75, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 75, 75, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 75, 75, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 75, 75, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 75, 75, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 75, 75, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 75, 75, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 75, 75, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 75, 75, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 654,804,232 (2.44 GB)

 Trainable params: 210,405,506 (802.63 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 420,811,014 (1.57 GB)

None


In [ ]:
# Assuming the second last layer is the one before the final classification layer
# model_1 = Model(inputs=model1.input, outputs=model1.layers[-2].output)
# model_2 = Model(inputs=model2.input, outputs=model2.layers[-2].output)

model1.load_weights('/content/model11_weights2.weights.h5')
model_intermediate_output = model1.layers[-2].output

model_1= Model(inputs=model1.input, outputs=model_intermediate_output)

#model2.load_weights('/content/model22_weights2.weights.h5')
model_intermediate_output = model2.layers[-2].output

model_2= Model(inputs=model2.input, outputs=model_intermediate_output)

hf_model.load_weights('/content/model33_weights2.weights.h5')
model_intermediate_output = hf_model.layers[-2].output

model_3= Model(inputs=hf_model.input, outputs=model_intermediate_output)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Freeze all layers in the pre-trained models
for layer in model_1.layers:
    layer.trainable = False

for layer in model_2.layers:
    layer.trainable = False

for layer in model_3.layers:
    layer.trainable = False

In [ ]:
# Ensemble model
from tensorflow.keras.layers import Input, Add, Dense, Dropout
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2

# Create a shared input layer
shared_input = Input(shape=(300, 300, 3))

# Pass the shared input through each model
output_1 = model_1(shared_input)
output_2 = model_2(shared_input)
output_3 = model_3(shared_input)

# Concatenate the outputs from the models
ensemble_outputs = Concatenate()([output_1, output_2, output_3])

# Add additional layers for the ensemble
ensemble_outputs = Dense(64, activation='relu',kernel_regularizer=l2(0.01))(ensemble_outputs)
ensemble_outputs = Dropout(0.5)(ensemble_outputs)
ensemble_outputs = Dense(2, activation='softmax')(ensemble_outputs)

# Create the ensemble model
ensemble_model = Model(inputs=shared_input, outputs=ensemble_outputs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True,)

In [ ]:


# Compile the ensemble model
ensemble_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[CategoricalAccuracy(name='accuracy')]
)

# Summary of the ensemble model
ensemble_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 300, 300, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_4 (Functional) │ (None, 128)            │     19,645,151 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_5 (Functional) │ (None, 256)            │    233,960,064 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_6 (Functional) │ (None, 128)            │        923,264 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 512)            │              0 │ functional_4[0][0],    │
│                           │                        │                │ functional_5[0][0],    │
│                           │                        │                │ functional_6[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 64)             │         32,832 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 64)             │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 2)              │            130 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 254,561,441 (971.07 MB)

 Trainable params: 32,962 (128.76 KB)

 Non-trainable params: 254,528,479 (970.95 MB)

In [ ]:
history = ensemble_model.fit(
    train_generator,
    epochs=100,
    steps_per_epoch=20,
    validation_data=test_generator,
    shuffle=True,
    callbacks=[early_stopping]
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 512ms/step - accuracy: 0.9872 - loss: 0.1454 - val_accuracy: 0.8276 - val_loss: 0.5387
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 376ms/step - accuracy: 0.9968 - loss: 0.1208 - val_accuracy: 0.7931 - val_loss: 0.6609
Epoch 3/100
 5/20 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 0.9787 - loss: 0.1724

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.9384 - loss: 0.2261 - val_accuracy: 0.8276 - val_loss: 0.6175
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 29s 445ms/step - accuracy: 0.9796 - loss: 0.1676 - val_accuracy: 0.8276 - val_loss: 0.5450
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 360ms/step - accuracy: 1.0000 - loss: 0.1063 - val_accuracy: 0.8506 - val_loss: 0.5570
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 230ms/step - accuracy: 1.0000 - loss: 0.0925 - val_accuracy: 0.8161 - val_loss: 0.6356
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 384ms/step - accuracy: 0.9968 - loss: 0.1157 - val_accuracy: 0.8851 - val_loss: 0.4538
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 423ms/step - accuracy: 0.9905 - loss: 0.1343 - val_accuracy: 0.8161 - val_loss: 0.7024
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 180ms/step - accuracy: 0.9732 - loss: 0.1702 - val_accuracy: 0.7931 - val_loss: 0.5954
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 442ms/step - accuracy: 0.9828 - loss: 0.1191 - val_accuracy: 0.7

In [ ]:
test_loss, test_accuracy = ensemble_model.evaluate( test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 323ms/step - accuracy: 0.8959 - loss: 0.4750


In [ ]:
test_loss, test_accuracy = hf_model.evaluate( test_generator)

 6/11 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.9028 - loss: 0.3532

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 434ms/step - accuracy: 0.8588 - loss: 0.4074


In [ ]:
test_loss, test_accuracy = model1.evaluate( test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.5507 - loss: 0.9115


In [ ]:
test_loss, test_accuracy = model2.evaluate( test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.8960 - loss: 0.2862


In [ ]:
ensemble_model.save_weights("ensemble_weights2.weights.h5")

#voting ensemble(incomplete)

In [ ]:
import numpy as np
hf_model.predict(test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step


array([[-0.75110996, -0.7166433 ,  0.7416833 , ..., -0.71823096,
         0.51210517, -0.6537259 ],
       [ 1.2479135 , -0.7541948 ,  0.8904833 , ..., -0.71823096,
         0.5913039 ,  2.3909254 ],
       [ 1.0720742 , -0.7541948 , -0.6729674 , ...,  2.9772568 ,
         2.596955  , -0.6537259 ],
       ...,
       [-0.16272539, -0.7541948 ,  3.0058444 , ..., -0.71823096,
        -0.66143495, -0.6537259 ],
       [ 1.7089156 , -0.7541948 ,  1.5230234 , ...,  0.2627018 ,
        -0.2515838 , -0.28042603],
       [-0.18909997, -0.2207312 , -0.40541375, ..., -0.71823096,
         1.8253269 , -0.6537259 ]], dtype=float32)

In [ ]:
import numpy as np

# Predict using each model
preds_model_1 = np.argmax(model1.predict(test_generator),axis=1)
preds_model_2 = np.argmax(model2.predict(test_generator),axis=1)
preds_model_3 = model_3.predict(test_generator)

# Convert sparse predictions to dense if needed
# (Assuming predictions are probabilities and not sparse tensors)
# If the predictions are already dense, you can skip this step
preds_model_1_dense = np.array(preds_model_1)
preds_model_2_dense = np.array(preds_model_2)
preds_model_3_dense = np.array(preds_model_3)

# Convert predictions to class labels
labels_model_1 = np.argmax(preds_model_1_dense, axis=1)
labels_model_2 = np.argmax(preds_model_2_dense, axis=1)
labels_model_3 = np.argmax(preds_model_3_dense, axis=1)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
2

KeyboardInterrupt: 

In [ ]:
ensemble_preds = (preds_model_1 + preds_model_2 + preds_model_3) / 3


In [ ]:
weights = [0.4, 0.3, 0.3]  # Example weights
ensemble_preds = (weights[0] * preds_model_1 +
                  weights[1] * preds_model_2 +
                  weights[2] * preds_model_3)


In [ ]:
from scipy.stats import mode

# Convert probabilities to class labels
preds_class_1 = np.argmax(preds_model_1, axis=1)
preds_class_2 = np.argmax(preds_model_2, axis=1)
preds_class_3 = np.argmax(preds_model_3, axis=1)

# Stack the predictions and use mode to find the most common class
all_preds = np.stack([preds_class_1, preds_class_2, preds_class_3], axis=0)
ensemble_preds = mode(all_preds, axis=0)[0][0]


In [ ]:
from sklearn.metrics import accuracy_score

# Assuming y_test contains the true labels
accuracy = accuracy_score(y_test, ensemble_preds)
print(f'Ensemble Accuracy: {accuracy:.4f}')